IMPLEMENTING DECISION TREES FOR CLASSIFICATION EGS


In [4]:
import numpy as np

In [26]:
x_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])

y_train = np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

#each col in x_train represents a feature, namely, ear shape(1: pointy, 0:floppy), face shape(1:round, 0:not), 
#whiskers(1:present, 0:absent) and the y_train represents if the sample is a cat or no where 1 is a cat

m = x_train.shape[0]
n = x_train.shape[1]

In [15]:
#the entropy func
def entropy(p):
    """
    Returns the entropy of a certain node with positive fraction p
    
    Args:
        p(int): positive fraction of a node

    Returns:
        returns entropy
    """
    if p ==0 or p == 1:
        return 0
    else:
        return (-p*np.log2(p)-(1-p)*np.log2(1-p))

In [98]:
#splitting function based on a certain feature whose index is present as a param
def splitting(x, feature_index):
    """
    Returns the indices of samples which are in the left and right node of the split

    Args:
        x(ndarray (m,n)): numpy array containing the training samples of the initial node
        feature_index (int): the index of the feature based on which the split is to be done, 0 <= feature_index <=n

    Returns:
        left_indices (list): index of the features wrt x array which are in the left node
        right_indices (right): index of the features wrt x array which are in right node
    """

    left_indices = []   #positive classes always go to the left node
    right_indices = []
    for i in range(len(x)):
        if x[i, feature_index] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices, right_indices

In [62]:
#weighted sum of entropies for a layer linked to a node
def weighed_sum(x, y, left_indices, right_indices):
    """
    Returns the weighted sum of the new two nodes getting split from the initial node whose elements are x, y
    
    Args:
        x(ndarray (m,n)): numpy array which is the feature set of the initial node
        y(ndarray (m,)): target vector of the initial node
        left_indices(list): list of indices in the new left split from the initial node wrt the original feature index
        right_indices(list): list of indices in the new right split from the initial node wrt the original feature index

    Returns:
        weighted sum of entropy for the split
    """
    w_left = len(left_indices)/len(x)
    w_right = len(right_indices)/len(x)

    if len(left_indices) != 0:
        p_left = sum(y[left_indices])/len(left_indices)
    else:
        p_left = 0
    
    if len(right_indices) != 0:
        p_right = sum(y[right_indices])/len(right_indices)
    else:
        p_right = 0

    H_left = entropy(p_left)
    H_right = entropy(p_right)
    return (w_left*H_left + w_right*H_right)

In [18]:
#finding info gain of a node
def info_gain(x, y, left_indices, right_indices):
    """
    Returns the information gain of the new split from the initial node

    Args:
        x(ndarray (m,n)): numpy array which is the feature set of the initial node
        y(ndarray (m,)): target vector of the initial node
        left_indices(list): list of indices in the new left split from the initial node wrt the original feature index
        right_indices(list): list of indices in the new right split from the initial node wrt the original feature index

    Returns:
        information gain of the new split
    """
    w_sum = weighed_sum(x, y, left_indices, right_indices)
    p_node = sum(y)/len(y)
    H_node = entropy(p_node)

    return H_node - w_sum

In [52]:
def finding_best_split(x, y, feature_indices_list):  #feature_indices_list in this 
    #case will be [0, 1, 2]

    """
    Returns the best feature to split a node with feature set x

    Args:
        x(ndarray (m,n)): the feature set of the node we are at currently
        y(ndarray (m,)): the target vector of the node we are currently at
        feature_indices_list(list): list containing the indices of the features

    Returns:
        best_feature_index (int): the index of the best feature to split that node upon

    
    """
    ig = 0
    best_feature_index = 0
    for i in feature_indices_list:
        l_list, r_list = splitting(x, i)
        ig_ = info_gain(x, y, l_list, r_list)
        if ig_ > ig:
            ig = ig_
            best_feature_index = i
    
    return best_feature_index

In [53]:
best = finding_best_split(x_train, y_train, [0,1,2])
print(best) 
#gives the index 2 indicting the root node shld split based on the "solitary" feature or the 3rd feature

2


In [54]:
l, r = splitting(x_train, best)
print(l, r)
print(info_gain(x_train, y_train, l, r))

[0, 1, 4, 5, 7] [2, 3, 6, 8, 9]
1.0


In [58]:
print(x_train[l])
print(l)

[[1 1 1]
 [1 0 1]
 [1 1 1]
 [0 1 1]
 [1 0 1]]
[0, 1, 4, 5, 7]


In [63]:
l_, r_ = splitting(x_train[l], 2)  #when best index for feature is 2 then r list is empty
print(l_, r_)  #these indices are now based on x_train[l] sub array
print(x_train[l][l_])

[0, 1, 2, 3, 4] []
[[1 1 1]
 [1 0 1]
 [1 1 1]
 [0 1 1]
 [1 0 1]]


In [64]:
best = finding_best_split(x_train[l], y_train[l], [0,1,2])
print(best) 

0


In [36]:
l0, r0 = splitting(x_train, 0)
print(l0, r0)
print(info_gain(x_train, y_train, l0, r0))

[0, 1, 2, 3, 4, 7, 9] [5, 6, 8]
0.034851554559677034


In [37]:
l1, r1, = splitting(x_train, 1)
print(l1, r1)
print(info_gain(x_train, y_train, l1, r1))

[0, 4, 5, 8] [1, 2, 3, 6, 7, 9]
0.12451124978365313


In [93]:
print(x_train[[]])

[]


In [111]:
def DT(depth, reverse, max_depth, node_list, waiting_list, x, y, l, r):
    while 0 <= depth < max_depth:
        if reverse == 0:
            depth += 1

        if reverse == 0:
            if len(l) != 0:
                x = x_train[l]
            else:
                x = x_train
            if len(r) != 0:
                y = y_train[l]
            else:
                y = y_train
        
        # if reverse == 1:
        #     x = x_train[r]


        best = finding_best_split(x, y, [0,1,2])
        l,r = splitting(x, best)

        if depth != max_depth:
            waiting_list.append(('R', depth, r))

        node_list.append(('L', depth, l))

        if depth == max_depth:
            node_list.append(('R', depth, r))

        # ig = info_gain(x, y, l, r)
        # if ig <= 0.03:
        #     break

        # if reverse == 0:
        #     x = x_train[l]
        #     y = y_train[l]


        # if depth == 3:
        #     reverse = 1
        #     depth -= 1

            

In [119]:
depth = 0
waiting_list = []
node_list = []
feature_indices_list = [0,1,2]
x = x_train
y = y_train
reverse = 0
l = []
r = []

DT(depth, reverse, 3, node_list, waiting_list, x, y, l, r)
print(node_list)
print(waiting_list)

[('L', 1, [0, 1, 4, 5, 7]), ('L', 2, [0, 1, 2, 4]), ('L', 3, [0, 1, 3]), ('R', 3, [2])]
[('R', 1, [2, 3, 6, 8, 9]), ('R', 2, [3])]


In [125]:
#applying splits for right node of first split
depth_ = 0
node_list_ = []
waiting_list_ = []
l_ = []
r_ = []
x_ = x_train[waiting_list[0][2]]
y_ = y_train[waiting_list[0][2]]
print(y_)
#DT(depth_, reverse, 3, node_list_, waiting_list_, x_train[waiting_list[0][2]], y_train[waiting_list[0][2]], l_, r_)
print(node_list_)

[0 0 0 0 0]
[]
